In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy
import itertools
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


In [28]:
def find_entry_int_distance(coordinates, piece: intervals.main_objs.ImportedPiece):
    tone_list = []
    all_tones = piece.getNoteRest()
    
    for item in coordinates:
        filtered_tones = all_tones.loc[item] 
        tone_list.append(filtered_tones)
        
    noteObjects = [note.Note(tone) for tone in tone_list]
    _ints = [interval.Interval(noteObjects[i], noteObjects[i + 1]) for i in range(len(noteObjects) - 1)]
    entry_ints = []
    
    for _int in _ints:
        entry_ints.append(_int.directedName)
    
    return entry_ints

def split_by_threshold(seq, max_diff=70):  
    it = iter(seq)
    last = next(it)
    part = [last]

    for curr in it:
        if curr - last > max_diff:
            yield part
            part = []

        part.append(curr)
        last = curr
#         print(part)
        
    yield part
    

def classify_by_offset(offset_diffs):
    """
    Put logic for classifying an offset list here
    """
    #
#     offset_difference_list = points['Time_Entry_Intervals']

    alt_list = offset_diffs[::2]

    if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
        return 'PEN'
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
    elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
        return 'ID'
    elif len(offset_diffs) >= 1:
        return 'FUGA'


In [29]:

piece = importScore('https://crimproject.org/mei/CRIM_Model_0015.mei')


Memoized piece detected.


In [33]:
# THIS WORKS 3/1/22
# piece = importScore('https://crimproject.org/mei/CRIM_Model_0008.mei')
points = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=5)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=5)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
#     voice_list = entry_array['voice'].to_list()
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    for item in split_list:
#         print(item)
        array = entry_array[entry_array.index.get_level_values(0).isin(item)]
        short_offset_list = array.index.to_list()
        time_ints = numpy.diff(array.index).tolist()
        voice_list = array['voice'].to_list()
# time_ints
        tone_coordinates =  list(zip(short_offset_list, voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        first_offset = short_offset_list[0]
# #     first_note_list = nr[offset_list]
        meas_beat = det[det.index.get_level_values('Offset').isin(short_offset_list)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()


        temp = {'First_Offset': first_offset, 
                    'Offsets': short_offset_list, 
                    'Measures_Beats': meas_beat_list,
                    "Soggetti": matches,
                    'Voices': voice_list, 
                    'Time_Entry_Intervals': time_ints, 
                    'Melodic_Entry_Intervals': mel_ints}
        
        points = points.append(temp, ignore_index=True).sort_values("First_Offset")
        points = points[points['Offsets'].apply(len) > 1]
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
points


# points_no_singles = points_clean['Soggetti'].apply(lambda x: ('\n'.join(x)))



,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type
0,0.0,"[1/1.0, 3/1.0, 10/1.0, 12/2.0]","[P-4, P-5, P-4]","[0.0, 16.0, 72.0, 90.0]","[5, -2, 2, 2, 2, 4, -2, 2, 2, 2]","[16.0, 56.0, 18.0]","[Superior, Contratenor, Tenor, Bassus]",FUGA
1,38.0,"[5/4.0, 7/2.0, 7/4.0, 10/4.0, 14/4.0, 16/4.0, ...","[P-5, P-4, P8, P-8, P5, P4, P-8]","[38.0, 50.0, 54.0, 78.0, 110.0, 126.0, 138.0, ...","[-3, 2, -2, -2, 2, -3, 2, -2, -2, 1]","[12.0, 4.0, 24.0, 32.0, 16.0, 12.0, 8.0]","[Superior, Contratenor, Bassus, Superior, Teno...",FUGA
2,174.0,"[22/4.0, 24/2.0, 26/4.0, 28/2.0]","[P5, P-12, P5]","[174.0, 186.0, 206.0, 218.0]","[3, -2, -2, -2, -3, 3, -2, -2, -2, -2]","[12.0, 20.0, 12.0]","[Contratenor, Superior, Bassus, Tenor]",ID
3,266.0,"[34/2.0, 38/2.0, 40/2.0]","[P1, P-5]","[266.0, 298.0, 314.0]","[2, -3, 5, 1, -5]","[32.0, 16.0]","[Bassus, Tenor, Bassus]",FUGA
4,274.0,"[35/2.0, 36/2.0, 41/2.0]","[P-5, P1]","[274.0, 282.0, 322.0]","[2, -3, 5, 1, -2]","[8.0, 40.0]","[Superior, Contratenor, Tenor]",FUGA
5,342.0,"[43/4.0, 48/1.0]",[P-8],"[342.0, 376.0]","[1, 1, 2, -2, 4]",[34.0],"[Superior, Tenor]",FUGA
6,366.0,"[46/4.0, 51/2.0, 53/2.0, 54/2.0, 54/4.0]","[P-8, P12, P-8, P4]","[366.0, 402.0, 418.0, 426.0, 430.0]","[1, -2, -2, -2, -2]","[36.0, 16.0, 8.0, 4.0]","[Contratenor, Bassus, Superior, Bassus, Tenor]",FUGA
7,446.0,"[56/4.0, 58/2.0, 60/4.0, 61/4.0, 63/2.0, 64/2....","[P5, P-8, P-5, P5, P8, P-8]","[446.0, 458.0, 478.0, 486.0, 498.0, 506.0, 518.0]","[4, 2, 2, -3, 3]","[12.0, 20.0, 8.0, 12.0, 8.0, 12.0]","[Contratenor, Superior, Tenor, Bassus, Contrat...",FUGA
8,534.0,"[67/4.0, 70/2.0, 74/2.0]","[P5, P-5]","[534.0, 554.0, 586.0]","[-2, -2, -2, 2, -2]","[20.0, 32.0]","[Bassus, Contratenor, Bassus]",FUGA
9,538.0,"[68/2.0, 72/2.0, 74/4.0, 76/3.0, 77/4.0, 79/4....","[P-4, P4, P8, P-8, m-7, P4, P8, P4, P-8, P8]","[538.0, 570.0, 590.0, 604.0, 614.0, 630.0, 634...","[-2, -2, -2, 2, 2, -2, -2, -3, 2, 2]","[32.0, 20.0, 14.0, 10.0, 16.0, 4.0, 8.0, 64.0,...","[Superior, Tenor, Tenor, Superior, Contratenor...",FUGA


In [32]:
# Durations of Notes in Ngrams
#  KEEP THIS!
nr = piece.getNoteRest()
dur = piece.getDuration(df=nr)
ng = piece.getNgrams(df=dur, n=5)
ng

,Superior,Contratenor,Tenor,Bassus
0.0,"(4.0, 6.0, 2.0, 2.0, 2.0)","(16.0, 4.0, 6.0, 2.0, 2.0)","(72.0, 4.0, 6.0, 2.0, 2.0)","(54.0, 2.0, 2.0, 4.0, 2.0)"
4.0,"(6.0, 2.0, 2.0, 2.0, 3.0)",NaN,NaN,NaN
10.0,"(2.0, 2.0, 2.0, 3.0, 0.5)",NaN,NaN,NaN
12.0,"(2.0, 2.0, 3.0, 0.5, 0.5)",NaN,NaN,NaN
14.0,"(2.0, 3.0, 0.5, 0.5, 2.0)",NaN,NaN,NaN
...,...,...,...,...
1316.0,NaN,"(1.0, 1.0, 2.0, 2.0, 2.0)","(3.0, 1.0, 2.0, 2.0, 4.0)","(2.0, 1.0, 1.0, 1.0, 1.0)"
1317.0,NaN,"(1.0, 2.0, 2.0, 2.0, 4.0)",NaN,NaN
1318.0,NaN,"(2.0, 2.0, 2.0, 4.0, 16.0)",NaN,"(1.0, 1.0, 1.0, 1.0, 2.0)"
1319.0,NaN,NaN,"(1.0, 2.0, 2.0, 4.0, 16.0)","(1.0, 1.0, 1.0, 2.0, 4.0)"
